# 머신 러닝 교과서 - 파이토치편

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-pytorch/blob/main/ch15/ch15_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 패키지 버전 체크

check_packages.py 스크립트에서 로드하기 위해 폴더를 추가합니다:

권장 패키지 버전을 확인하세요:

In [1]:
from python_environment_check import check_packages


d = {
    'torch': '1.8.0',
}
check_packages(d)

[OK] Your Python version is 3.13.11 | packaged by Anaconda, Inc. | (main, Dec 10 2025, 21:21:58) [MSC v.1929 64 bit (AMD64)]
[OK] torch 2.9.1+cpu


# 15장 - 순환 신경망으로 순차 데이터 모델링 (파트 1/3)

**목차**

- 순차 데이터 소개
  - 순차 데이터 모델링: 순서를 고려한다
  - 순차 데이터 vs 시계열 데이터
  - 시퀀스 표현
  - 시퀀스 모델링의 종류
- 시퀀스 모델링을 위한 RNN
  - RNN의 반복 구조 이해
  - RNN의 활성화 출력 계산
  - 은닉 순환과 출력 순환
  - 긴 시퀀스 학습의 어려움
  - LSTM 셀

In [2]:
import matplotlib
from IPython.display import Image

# 15.1 순차 데이터 소개

## 15.1.1 순차 데이터 모델링: 순서를 고려한다

- 지도 학습 알고리즘의 입력 데이터: 독립 동일 분포(independent and identically distributed)

 > traing samples이 상호 독립적이고 같은 분포에 속한다.

 > 붓꽃 데이터 x1,x2, .., xn이 samples일 때 각 샘플은 상호 독립적

- sequence: 시간 순서가 있다


## 15.1.2 순차 데이터 vs 시계열 데이터

- 시계열 데이터(time series) - 연속적인 timestamp를 갖는 데이터

- 순차 데이터: 텍스트 데이터, DNA sequence


## 15.1.3 시퀀스 표현

- sequence 표현: <x1,x2, .., xT> T는 시퀀스 길이

- RNN은 시퀀스 데이터의 모델링 

In [4]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_01.png', width=500)

## 15.1.4 시퀀스 모델링의 종류

- 응용: 언어 번역, image captioning

In [5]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_02.png', width=500)

1. 다대일: 입력 텍스트 - 출력 감성분석 레이블

2. 일대다: 입력: 이미지, 출력 이미지 내용을 요약한 문장

3. 다대다:

 - 동기적인 다대다 모델링: 입력 비디오 프레임, 출력 각 프레임의 레이블

 - 지연 다대다 모델; 언어 번역

# 15.2 시퀀스 모델링을 위한 RNN

## 15.2.1 RNN의 반복 구조 이해

In [6]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_03.png', width=500)

In [7]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_04.png', width=500)

## 15.2.2 RNN의 활성화 출력 계산

In [8]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_05.png', width=500)

In [9]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_06.png', width=700)

왜 𝑜𝑡를 출력 결과로 안 쓰는가?
```Python
x1 → x2 → x3 → x4 → x5
           ↓
         최종 출력
```
- 감성분석: 최종 출력은 전체 시퀀스를 요약한 결과: 마지막 hidden state hT를 사용 > 핵심 사항으로 반드시 이해해야 한다

 > RNN은 매 시점마다 출력 𝑜𝑡가 나오는데, 
 
 > 왜 감성분석 같은 문제에서는 중간 𝑜𝑡들을 안 쓰고 마지막(최종) 출력만 쓰는가?”

   + 핵심은 문제의 “정답(label)이 어디에 붙어 있느냐” .

RNN에서 ht, ot가 무엇인가?

- ht: t 시점까지 읽은 내용을 요약한 메모: ht = f(ht-1, xt)

 > hT: 마지막 은닉 상태 hT는 x1,x2, ..,xT를 모두 읽고 요약 상태

- ot: ht로 만든 출력 > ot = g(ht)

In [ ]:
output, (h, c) = lstm(x) # 655 페이지 코드
# output: 모든 시점의 o_t
# h: 마지막 hidden state

 #- output를 버리고 h만 사용

RNN의 기억은 ‘복사(ht-1) + 약간의 수정(xt)’을 반복

 - 복사 강도를 결정하는 손잡이: 𝑊ℎ

 - 역전파는 Wh를 시간만큼 계속 곱한다.

  > Wh가 1보다 작으면 기억이 사라지고, 너무 크면 폭발한다. >> 반드시 이해해야 하는 핵심 포인트 

역전파: 현재 손실이 과거 손실에 얼마나 기인하나?

 - ∂𝐿/∂ℎ𝑡

 > 𝑡 시점의 기억이 최종 손실에 얼마나 기여했나?

시간 역전파가 생기는 이유:

- h1 > h2 > h3 > 최종 손실 L

  >  최종 손실 L에 h3가 영향 > h2가 영향 > h1이 영향

왜 책임이 전달될 때마다 W_h가 튀어나오나?

- ht = tanh(Wh.ht-1 + Wx.xt)

 > ∂ℎ𝑡/∂ℎ𝑡-1 = Wh

역전파: 손실이 과거 원인이 어디에 있나?

- h1 → h2 → h3 → 최종 결과 L

 > 손실 L의 원인 > h3 > h2 > h1 순서로 

손실함수 L = l(yi, yi^)

 - yi^ = f(h3)

 - L = f(h3) = tanh(h3)으로 단순화 표현

- ∂𝐿/∂ℎ1 = ∂𝐿/∂ℎ3 x  ∂h3/∂ℎ2 x ∂h2/∂ℎ1 

-> h1의 미세 변화가 최종 손실 L에 얼마나 변하나

 > ∂hk/∂ℎk-1 ≈ Wh

 > ∂𝐿/∂ℎ1 ≈ ∂𝐿/∂ℎ3 x Wh x Wh

 > L = tanh(h3) = f(h3), h3 = tanh(h2) = g(h2), h2 = tanh(h1) = k(h1)

   >> L = tanh(tanh(tanh(h1))) = f(g(k(h1)))

   >> dL/dh1 = dL/dh3 . dh3/dh2 . dh2/dh1

기울기 소실 문제:
 
 - Wh = 0.9 > ∂𝐿/∂ℎ3 x 0.9 x 0.9

  > 100번 → 사실상 0

  > 기울기 소실 (vanishing gradient)

기울기 폭주 (exploding gradient):

 - Wh = 1.1

  > 100번 > 증폭

ht = tanh(Wh.ht-1 + Wx.xt)의 미분:

- tanh'(x) = 1 - tanh^2(x) > 항상 1보다 작은 0.2 ~ 0.8 사이

- Wh x (0.x) x Wh x (0.x) x ...

 > 기울기 소실이 더 빨라짐

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

# 차원
B = 2          # batch
I = 3          # input_size
H = 4          # hidden_size가 5,6,7 등으로 지정
# h0: 진폭 변화, h1: 저주파 패턴, h2: 고주파 패턴 등의 표현
C = 5          # num_classes

# 한 스텝 입력/이전 은닉
x_t = torch.randn(B, I)     # (B, I)
h_prev = torch.randn(B, H)  # (B, H)

# 파라미터를 "수식처럼" 직접 만들기
W_xh = torch.randn(H, I, requires_grad=True)  # (H, I)
W_hh = torch.randn(H, H, requires_grad=True)  # (H, H)
b_h  = torch.randn(H, requires_grad=True)     # (H,)

W_hy = torch.randn(C, H, requires_grad=True)  # (C, H)
b_y  = torch.randn(C, requires_grad=True)     # (C,)

# ----- forward (한 스텝) -----
a_t = x_t @ W_xh.T + h_prev @ W_hh.T + b_h    # (B, H)
h_t = torch.tanh(a_t)                         # (B, H)
logits = h_t @ W_hy.T + b_y                   # (B, C)
y = torch.tensor([1, 3])                      # 정답 라벨 (B,)
loss = F.cross_entropy(logits, y)

# ----- backward -----
loss.backward()

print("loss:", loss.item())
print("grad W_hh norm:", W_hh.grad.norm().item())
print("grad W_xh norm:", W_xh.grad.norm().item())
print("grad h_prev (through time) exists?:", h_prev.requires_grad)


loss: 5.122039794921875
grad W_hh norm: 0.03478823974728584
grad W_xh norm: 0.06618761271238327
grad h_prev (through time) exists?: False


## 은닉 순환과 출력 순환

In [10]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_07.png', width=500)

In [4]:
import torch
import torch.nn as nn

torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh 크기:', w_xh.shape)
print('W_hh 크기:', w_hh.shape)
print('b_xh 크기:', b_xh.shape)
print('b_hh 크기:', b_hh.shape)

W_xh 크기: torch.Size([2, 5])
W_hh 크기: torch.Size([2, 2])
b_xh 크기: torch.Size([2])
b_hh 크기: torch.Size([2])


In [5]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

## 간단한 RNN의 출력:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

## 수동으로 출력 계산
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'타임 스텝 {t} =>')
    print('   입력           :', xt.numpy())

    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print('   은닉           :', ht.detach().numpy())

    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))
    # ot = tanh(xtWxh +bxh + ht-1Whh + bhh)
    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    
    # rnn 출력과 수동 계산 결과를 비교
    print('   출력 (수동)     :', ot.detach().numpy())
    print('   RNN 출력       :', output[:, t].detach().numpy())
    print()

타임 스텝 0 =>
   입력           : [[1. 1. 1. 1. 1.]]
   은닉           : [[-0.4701929  0.5863904]]
   출력 (수동)     : [[-0.3519801   0.52525216]]
   RNN 출력       : [[-0.3519801   0.52525216]]

타임 스텝 1 =>
   입력           : [[2. 2. 2. 2. 2.]]
   은닉           : [[-0.88883156  1.2364397 ]]
   출력 (수동)     : [[-0.68424344  0.76074266]]
   RNN 출력       : [[-0.68424344  0.76074266]]

타임 스텝 2 =>
   입력           : [[3. 3. 3. 3. 3.]]
   은닉           : [[-1.3074701  1.886489 ]]
   출력 (수동)     : [[-0.8649416   0.90466356]]
   RNN 출력       : [[-0.8649416   0.90466356]]



## 15.2.4 긴 시퀀스 학습의 어려움
- 그래디언트 소실 : vanishing gradient

- 그래디언트 폭발: exploding gradient

In [13]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_08.png', width=500)

## 15.2.5 LSTM 셀

- 그래디언트 소실 문제를 해결: 메모리 셀 사용

 > 기본 RNN은 기억을 ‘곱해서’ 전달:

   ht = tanh(Wh.ht-1 + ...)

   역전파: gradient x Wh x tanh'(.) x Wh x tanh'(.) x ...
 
 > LSTM은 기억을 ‘그대로 흘려보내는 통로(덧셈 경로)’를 만들어서 그래디언트가 사라지지 않게 한다.

   ct​=ft​⊙ct−1​+it​⊙c~t​ : 그대로 더해서 전달


LSTM에는 3개의 스위치가 있다.

1. Forget gate : ft

  → “이 기억, 계속 가져갈까?”

  > 중요하면: 거의 그대로 유지

  > 필요 없으면: 조금씩 지움

  > 그래서 이름이 Forget Gate

2. Input gate : 𝑖𝑡	​

  → “지금 들어온 정보 중에서 기억으로 남길 건 뭐야?”

  > 중요한 정보만 일부 기록, 쓸모없으면 무시

  > 그래서 Input Gate

3. Output gate : 𝑜𝑡

  → “기억 중 뭐를 밖으로 보여줄까?”

  > 기억 전체를 다 쓰지 않음, 필요한 부분만 꺼냄

  > 그래서 Output Gate

 => 이 스위치들이 그래디언트가 지나갈 길을 관리합니다.

  + 기억 전달 = 유지 + 추가 (덧셈 구조)

  + 덧셈은 미분해도 1

  + 그래서 그래디언트가 줄지 않고 살아남음

LSTM을 한 문장으로 바꾸면

- “중요한 기억은 그대로 들고 가고, 필요 없는 기억만 지우고,

- 새 정보는 필요한 만큼만 추가한다.”

In [6]:
import torch
import torch.nn as nn

class ManualLSTMCell(nn.Module):
    """
    LSTM cell을 게이트 단위로 직접 구현:
    f_t, i_t, g_t(=c̃_t), o_t 를 계산하고
    c_t = f_t*c_{t-1} + i_t*g_t
    h_t = o_t*tanh(c_t)
    """
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # (x_t와 h_{t-1}를 합쳐 한 번에 처리) -> 4개의 게이트를 동시에 계산
        # gates = [f, i, g, o] 를 한 번에 뽑아냄
        self.W = nn.Linear(input_size + hidden_size, 4 * hidden_size)
        ## Wx .xt + Wh.ht-1 + b
        # 초기화는 PyTorch 기본으로 둬도 되고, 교육용으로 따로 설정해도 됨.

    def forward(self, x_t: torch.Tensor, h_prev: torch.Tensor, c_prev: torch.Tensor):
        """
        x_t:    (B, I)
        h_prev: (B, H) # 히든 상태
        c_prev: (B, H) # 셀 상태
        return: h_t, c_t, (f_t, i_t, g_t, o_t)
        """
        # 1) 입력과 이전 은닉을 붙인다 (concatenate)
        concat = torch.cat([x_t, h_prev], dim=1)  # (B, I+H)
        ## xt + ht-1
        
        # 2) 4H 차원으로 선형변환 후 4개로 쪼갠다 > 4개의 게이트를 한 번에 계산 
        gates = self.W(concat)                    # (B, 4H) - W =[Wx|Wh]
        ## gates = concat @ WT + b = [xt,ht-1]@WT + b
        f, i, g, o = gates.chunk(4, dim=1)        # 각각 (B, H)
        ## gates를 4개로 분리한다

        # 3) 게이트 활성화
        f_t = torch.sigmoid(f)    # forget gate
        i_t = torch.sigmoid(i)    # input gate
        g_t = torch.tanh(g)       # candidate (c̃)
        o_t = torch.sigmoid(o)    # output gate

        # 4) cell state 업데이트: 유지 + 추가 (LSTM의 핵심)
        c_t = f_t * c_prev + i_t * g_t

        # 5) hidden state 출력: cell의 일부를 밖으로 보여줌
        h_t = o_t * torch.tanh(c_t)

        return h_t, c_t, (f_t, i_t, g_t, o_t)


In [7]:
# LSTM 셀을 시간축으로 어떻게 반복하는가
# 하나의 LSTM 셀을 시간축으로 반복 호출하여 시퀀스를 처리하고,
# 각 시점의 hidden state와 마지막 상태를 반환하는
# LSTM의 시간 전개(unrolling)를 직접 구현한 코드

class ManualLSTM(nn.Module): # LSTM 전체 시퀀스 모델 (cell을 T번 반복)
    """
    ManualLSTMCell을 T번 반복해서 시퀀스를 처리.
    """
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.cell = ManualLSTMCell(input_size, hidden_size) #한 시점의 LSTM 연산을 담당하는 셀

    def forward(self, x: torch.Tensor, h0=None, c0=None):
        """
        x: (B, T, I)
        return:
          outputs: (B, T, H)  각 시점 h_t
          (h_T, c_T): 마지막 상태
        """
        B, T, I = x.shape # B: batch size, T: 시퀀스 길이 (time steps), I: input_size
        H = self.cell.hidden_size

        if h0 is None:
            h_t = torch.zeros(B, H, device=x.device)
        else:
            h_t = h0

        if c0 is None:
            c_t = torch.zeros(B, H, device=x.device)
        else:
            c_t = c0

        outputs = [] # 각 시점의 h_t를 모아둘 리스트, 마지막에 (B, T, H)로 합칠 예정
        gate_logs = []  # (선택) 게이트 값들을 모아서 관찰

        for t in range(T): # 시퀀스를 시간 축으로 반복
            x_t = x[:, t, :]  # (B, I) > t 번째 시점의 입력 추출
            h_t, c_t, gates = self.cell(x_t, h_t, c_t)
            outputs.append(h_t.unsqueeze(1))  # (B, 1, H)
            gate_logs.append(gates)

        outputs = torch.cat(outputs, dim=1)  # (B, T, H)
        return outputs, (h_t, c_t), gate_logs
        # outputs : 모든 시점의 hidden state > many-to-many 문제에 사용
        # (h_t, c_t): 마지막 시점 상태 > many-to-one 문제(감성분석 등)에 사용
        # gate_logs > 내부 동작 분석용 (선택)


In [8]:
def demo():
    torch.manual_seed(0)
    B, T, I, H = 1, 3, 5, 4

    x = torch.tensor([[[1.,1.,1.,1.,1.],
                       [2.,2.,2.,2.,2.],
                       [3.,3.,3.,3.,3.]]])  # (1,3,5)

    lstm = ManualLSTM(input_size=I, hidden_size=H)
    outputs, (hT, cT), gate_logs = lstm(x)

    print("outputs shape:", outputs.shape)  # (B,T,H)

    for t, (f_t, i_t, g_t, o_t) in enumerate(gate_logs):
        print(f"\n[t={t}]")
        print("forget f_t :", f_t.detach().numpy())
        print("input  i_t :", i_t.detach().numpy())
        print("cand   g_t :", g_t.detach().numpy())
        print("output o_t :", o_t.detach().numpy())
        print("h_t        :", outputs[:, t, :].detach().numpy())
        write = (i_t * g_t)              # 새로 쓴 기억 양
        show  = (o_t * torch.tanh(i_t * g_t))  # t=0에서 대략적인 출력 기여(직관용)

        print("write i*g:", write.detach().numpy())
        print("show :", show.detach().numpy())  # 원하면


if __name__ == "__main__":
    demo()


outputs shape: torch.Size([1, 3, 4])

[t=0]
forget f_t : [[0.31622654 0.31533563 0.54614383 0.389943  ]]
input  i_t : [[0.56318486 0.45477724 0.32914117 0.512559  ]]
cand   g_t : [[ 0.7574051   0.38519666  0.46551514 -0.20166232]]
output o_t : [[0.5244917 0.5081673 0.3149651 0.5226072]]
h_t        : [[ 0.21107733  0.08812051  0.04788488 -0.05382713]]
write i*g: [[ 0.4265591   0.17517868  0.15322019 -0.10336383]]
show : [[ 0.21107733  0.08812051  0.04788488 -0.05382713]]

[t=1]
forget f_t : [[0.22963709 0.1938902  0.54403424 0.27960595]]
input  i_t : [[0.6549426  0.49269423 0.24634506 0.61015356]]
cand   g_t : [[ 0.9637487   0.55198675  0.67223364 -0.15199715]]
output o_t : [[0.51577973 0.57470495 0.18223095 0.6029786 ]]
h_t        : [[ 0.32109734  0.17053013  0.0444533  -0.07298832]]
write i*g: [[ 0.6312001   0.27196068  0.16560143 -0.0927416 ]]
show : [[ 0.28825796  0.15255445  0.02990483 -0.05576142]]

[t=2]
forget f_t : [[0.15800612 0.11298788 0.53788704 0.19443546]]
input  i_t : [[

In [14]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_09.png', width=500)